# 第6课：将MCP聊天机器人连接到参考服务器 

现在你将扩展MCP聊天机器人的能力，使其能够连接到任何MCP服务器。 除了你在上一课构建的研究服务器工具外，你还会集成两个官方MCP服务器的工具。 

<img src="images/lesson_6.png" width="400">

## 开源MCP服务器

在这个[仓库](https://github.com/modelcontextprotocol/servers)中，你可以找到MCP服务器的参考实现集合，以及社区构建的服务器和额外资源的参考。 你将使用其中两个参考服务器，将它们的工具集成到你的MCP聊天机器人中：
- [fetch](https://github.com/modelcontextprotocol/servers/tree/main/src/fetch): 提供`fetch`工具，可以从互联网获取URL并将其内容提取为markdown格式。
- [filesystem](https://github.com/modelcontextprotocol/servers/tree/main/src/filesystem): 提供多个工具，用于与你指定的目录内的文件和目录进行交互。

你可以查看每个服务器的readme文件，了解它们暴露的功能以及如何运行它们。  

## 更新MCP聊天机器人 - 可选阅读

以下是你将对聊天机器人进行的更新。 如果你想了解更多代码细节，建议在观看视频之前或之后阅读本节。

- 聊天机器人不再硬编码服务器参数，而是从JSON文件读取服务器配置：
  ### 服务器配置
  在`L6/mcp_project`中，你可以找到`server_config.json`配置文件，其结构如下。
    ``` json
    {
        "mcpServers": {
            
            "filesystem": {
                "command": "npx",
                "args": [
                    "-y",
                    "@modelcontextprotocol/server-filesystem",
                    "."
                ]
            },
            
            "research": {
                "command": "uv",
                "args": ["run", "research_server.py"]
            },
            
            "fetch": {
                "command": "uvx",
                "args": ["mcp-server-fetch"]
            }
        }
    }
    ```
    对于参考服务器，命令`npx`和`uvx`会直接将服务器的文件安装到你的本地环境（你不需要提前安装）。 注意对于`filesystem`，`.`作为第三个参数提供，意思是"当前目录"。 这意味着你允许`fetch`服务器与当前目录内的文件和目录进行交互。

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> 访问 <code>mcp_project</code> 文件夹：</b> 1) 点击笔记本顶部菜单中的<em>"文件"</em>选项，然后 2) 点击<em>"打开"</em>，最后 3) 点击<em>L6</em>。
</div>

- 以下是更新后的MCP_chatbot的粗略示意图：
  
  <img src="images/updated_class.png" width="600">

  1. 不再只有一个会话，你现在有一个客户端会话列表，每个客户端会话与每个服务器建立一对一的连接；
  2. `available_tools`包含聊天机器人可以连接的所有服务器暴露的所有工具的定义。
  3. `tool_to_session`将工具名称映射到相应的客户端会话；这样，当LLM决定使用特定工具名称时，你可以将其映射到正确的客户端会话，从而使用该会话向正确的MCP服务器发送`tool_call`请求。
  4. `exit_stack`是一个上下文管理器，将管理mcp客户端对象及其会话，并确保它们被正确关闭。 在第5课中，你没有使用它，因为你使用了`with`语句，它在幕后使用了上下文管理器。 这里你可以再次使用`with`语句，但由于你有多个服务器需要连接，最终可能会使用多个嵌套的`with`语句。 `exit_stack`允许你动态添加mcp客户端及其会话，如下面的代码所示。
  5. `connect_to_servers`读取服务器配置文件，并为每个服务器调用辅助方法`connect_to_server`。 在后一个方法中，创建一个MCP客户端并用于将服务器作为子进程启动，然后创建一个客户端会话连接到服务器并获取服务器提供的工具列表的描述。
  6. `cleanup`是一个辅助方法，确保当你完成时所有连接都被正确关闭。 在第5课中，你依赖`with`语句自动清理资源。 这个cleanup方法有类似的目的，但针对你添加到exit_stack的所有资源；它按照添加的相反顺序关闭（你的MCP客户端和会话）——就像堆叠和卸载盘子一样。 这在网络编程中特别重要，可以避免资源泄漏。

## MCP聊天机器人的更新代码

In [ ]:
%%writefile mcp_project/mcp_chatbot.py

from dotenv import load_dotenv
from anthropic import Anthropic
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List, Dict, TypedDict
from contextlib import AsyncExitStack
import json
import asyncio

load_dotenv()

class ToolDefinition(TypedDict):
    name: str
    description: str
    input_schema: dict

class MCP_ChatBot:

    def __init__(self):
        # 初始化会话和客户端对象
        self.sessions: List[ClientSession] = [] # new
        self.exit_stack = AsyncExitStack() # new
        self.anthropic = Anthropic()
        self.available_tools: List[ToolDefinition] = [] # new
        self.tool_to_session: Dict[str, ClientSession] = {} # new


    async def connect_to_server(self, server_name: str, server_config: dict) -> None:
        """Connect to a single MCP server."""
        try:
            server_params = StdioServerParameters(**server_config)
            stdio_transport = await self.exit_stack.enter_async_context(
                stdio_client(server_params)
            ) # new
            read, write = stdio_transport
            session = await self.exit_stack.enter_async_context(
                ClientSession(read, write)
            ) # new
            await session.initialize()
            self.sessions.append(session)
            
            # List available tools for this session
            response = await session.list_tools()
            tools = response.tools
            print(f"\nConnected to {server_name} with tools:", [t.name for t in tools])
            
            for tool in tools: # new
                self.tool_to_session[tool.name] = session
                self.available_tools.append({
                    "name": tool.name,
                    "description": tool.description,
                    "input_schema": tool.inputSchema
                })
        except Exception as e:
            print(f"Failed to connect to {server_name}: {e}")

    async def connect_to_servers(self): # new
        """Connect to all configured MCP servers."""
        try:
            with open("server_config.json", "r") as file:
                data = json.load(file)
            
            servers = data.get("mcpServers", {})
            
            for server_name, server_config in servers.items():
                await self.connect_to_server(server_name, server_config)
        except Exception as e:
            print(f"Error loading server configuration: {e}")
            raise
    
    async def process_query(self, query):
        messages = [{'role':'user', 'content':query}]
        response = self.anthropic.messages.create(max_tokens = 2024,
                                      model = 'claude-3-7-sonnet-20250219', 
                                      tools = self.available_tools,
                                      messages = messages)
        process_query = True
        while process_query:
            assistant_content = []
            for content in response.content:
                if content.type =='text':
                    print(content.text)
                    assistant_content.append(content)
                    if(len(response.content) == 1):
                        process_query= False
                elif content.type == 'tool_use':
                    assistant_content.append(content)
                    messages.append({'role':'assistant', 'content':assistant_content})
                    tool_id = content.id
                    tool_args = content.input
                    tool_name = content.name
                    
    
                    print(f"Calling tool {tool_name} with args {tool_args}")
                    
                    # Call a tool
                    session = self.tool_to_session[tool_name] # new
                    result = await session.call_tool(tool_name, arguments=tool_args)
                    messages.append({"role": "user", 
                                      "content": [
                                          {
                                              "type": "tool_result",
                                              "tool_use_id":tool_id,
                                              "content": result.content
                                          }
                                      ]
                                    })
                    response = self.anthropic.messages.create(max_tokens = 2024,
                                      model = 'claude-3-7-sonnet-20250219', 
                                      tools = self.available_tools,
                                      messages = messages) 
                    
                    if(len(response.content) == 1 and response.content[0].type == "text"):
                        print(response.content[0].text)
                        process_query= False

    
    
    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")
        
        while True:
            try:
                query = input("\nQuery: ").strip()
        
                if query.lower() == 'quit':
                    break
                    
                await self.process_query(query)
                print("\n")
                    
            except Exception as e:
                print(f"\nError: {str(e)}")
    
    async def cleanup(self): # new
        """Cleanly close all resources using AsyncExitStack."""
        await self.exit_stack.aclose()


async def main():
    chatbot = MCP_ChatBot()
    try:
        # the mcp clients and sessions are not initialized using "with"
        # like in the previous lesson
        # so the cleanup should be manually handled
        await chatbot.connect_to_servers() # new! 
        await chatbot.chat_loop()
    finally:
        await chatbot.cleanup() #new! 


if __name__ == "__main__":
    asyncio.run(main())

## 运行MCP聊天机器人

**终端指令**

- 要打开终端，请运行下面的单元格。
- 导航到`mcp_project`目录：
    - `cd L6/mcp_project`
- 激活虚拟环境：
    - `source .venv/bin/activate`
- 运行聊天机器人：
    - `uv run mcp_chatbot.py`
- 要退出聊天机器人，请输入`quit`。
- 如果你运行了一些查询并想要访问`papers`文件夹或任何输出文件：1) 点击笔记本顶部菜单中的`文件`选项，然后 2) 点击`打开`，接着 3) 点击`L6` -> `mcp_project`。

In [ ]:
# 启动新终端
import os
from IPython.display import IFrame

IFrame(f"{os.environ.get('DLAI_LOCAL_URL').format(port=8888)}terminals/3", width=600, height=768)

请确保与聊天机器人进行交互。以下是一些查询示例：
- 获取此网站的内容：https://modelcontextprotocol.io/docs/concepts/architecture 并将内容保存到文件"mcp_summary.md"中，创建一个可视化图表总结"mcp_summary.md"的内容，并将其保存到文本文件中
- 获取deeplearning.ai并找到一个有趣的术语。围绕该术语搜索2篇论文，然后总结你的发现并将其写入名为results.txt的文件中

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>不同的运行结果：</b> AI聊天模型生成的输出可能因每次执行而异，这是由于它们的动态、概率性质。如果你的结果与视频中显示的不同，请不要惊讶。</p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>下载笔记本：</b> 1) 点击笔记本顶部菜单中的<em>"文件"</em>选项，然后 2) 点击<em>"下载为"</em>并选择<em>"笔记本(.ipynb)"</em>。</p>

<p> 📒 &nbsp; 如需更多帮助，请参阅<em>"附录 - 提示、帮助和下载"</em>课程。</p>

</div>

## 最后说明

如果你愿意，鼓励你重构`MCP_ChatBot`的代码：

- 如何异步连接到服务器
- 如果不同服务器的工具具有相同的名称怎么办
- 重新审视属性
  
以及你可能想到的任何其他想法。